In [47]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import time
import requests
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import numpy as np

## 텍스트 통계분석 패키지 ##
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# 말뭉치 분석 및 명사 추출 패키지 #
from konlpy.tag import Kkma       ; kkma = Kkma()
from konlpy.tag import Hannanum   ; hannanum = Hannanum()
from konlpy.tag import Okt        ; t = Okt()
from konlpy.tag import *
import pickle

import matplotlib.pyplot as plt

# 그림 옵션 지정 부분 #
plt.rcParams ['font.family'] = [ 'sans-serif']
plt.rcParams ['font.sans-serif'] = ['Malgun Gothic']
plt.rcParams["figure.figsize"] = (10,10)
plt.rcParams['axes.grid'] = False
csfont = {'fontname':'Malgun Gothic'}
# 그림 옵션 지정 부분 #

In [48]:
todaydate = datetime.datetime.today()

year = todaydate.strftime("%y")
month = todaydate.strftime("%m")
day = todaydate.strftime("%d")

In [39]:
url_origin = 'http://www.ssg.com/search.ssg?target=all&query=%EB%96%A1%EB%B3%B6%EC%9D%B4&ctgId=6000092899&ctgLv=1&page={}'

href_list = []


for i in range(1, 6):
    url = url_origin.format(i)
    driver = webdriver.Chrome(executable_path='chromedriver')
    driver.get(url)
    for q in range(1,3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    ee = soup.find('div', {'class': 'tmpl_itemlist'})
    s = ee.select('div.thmb')

    for q in s:
        x = q.find('a')
        ex = x.get('href')
        href_list.append(ex)
        
    time.sleep(2)    
    driver.close()



In [40]:
ch = []
data = {'name':ch, 'price':ch, 'review':ch,  'grade':ch, 'img_url':ch, 'product_num':ch, 'brand':ch, 'regi':ch, 'link': ch}
df = pd.DataFrame(data)

In [40]:
df

In [165]:
df.to_excel("rrr.xlsx")

In [41]:
def datas(soup):
    
    res = []

    name = soup.find('h2', {'class': 'cdtl_info_tit'})
    name_f = name.get_text()

    price = soup.find('em', {'class': 'ssg_price'}).get_text()
    price_f = int(re.sub(r'[^0-9]', '', price))
    
    img1 = soup.find('div', {'class': 'cdtl_item_image'})
    img_url = img1.find('img')
    img_url = img_url.get('src')
    
    review = 0
    grade = 0
    brand_f = np.NaN
    
    try:
        review = soup.find('em', {'id': 'gCommentTotalCnt'}).get_text() 
        review = int(re.sub(r'[^0-9]', '', review))
    except:
        print('리뷰정보 없음')
    
    try:
        grade = float(soup.find('em', {'class': 'cdtl_grade_total'}).get_text())
    except:
        print('별점정보 없음')
        
    check = driver.current_url    
    product_num = check.split('itemId=')[1]
    product_num = product_num.split('&')[0]
    
    
    
    
    try:
        brand = soup.find('span', {'class': 'cdtl_store_tittx'})
        brand_f = brand.get_text()
    except:
        print('브랜드정보 없음')
    
    
    
    res.append(name_f)
    res.append(price_f)
    res.append(review)
    res.append(grade)
    res.append(img_url)
    
    res.append(product_num)
    res.append(brand_f)
    res.append(np.NaN)
    return(res)

In [42]:
for i in range(0, len(href_list)):
    
    print(i)
    url = 'http://www.ssg.com' + href_list[i]
    driver = webdriver.Chrome(executable_path='chromedriver')
    driver.get(url)
    for q in range(1,3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')        
    res = datas(soup)
    res.append(driver.current_url)
    
    df.loc[len(df)] = res
    
    if(i >= 40):
        break
        
    driver.close()
    
    #time.sleep(2)

0
1
2
3
4
브랜드정보 없음
5
6
7
8
9
10
브랜드정보 없음
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
브랜드정보 없음
39
40
리뷰정보 없음
별점정보 없음


In [43]:
def names(x):
    res = False
    if '떡볶이' in x:
        res = True
    return res

df['check'] = df['name'].apply(names)
df1 = df[df['check'] == True]

In [44]:
def names2(x):
    res = x.rsplit('원산지', 1)[0]
    return(res)


df1['name'] = df1['name'].apply(names2)

C:\Users\saima\AppData\Local\Temp/ipykernel_13736/3014365492.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['name'] = df1['name'].apply(names2)


In [45]:
df1['year'] = year
df1['month'] = month
df1['day'] = day
df1['market'] = 'ssg'

C:\Users\saima\AppData\Local\Temp/ipykernel_13736/864266778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['year'] = year
C:\Users\saima\AppData\Local\Temp/ipykernel_13736/864266778.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['month'] = month
C:\Users\saima\AppData\Local\Temp/ipykernel_13736/864266778.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [25]:
df1.to_excel('ssg_.xls')

C:\Users\saima\AppData\Local\Temp/ipykernel_13736/2715718692.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df1.to_excel('ssg_.xls')


In [64]:
print(df1)

                          name    price   review  grade  \
0    [풀무원] 국물 떡볶이 423.5g (2인분)   3480.0  60929.0    4.9   
1            [노브랜드] 떡볶이 402.5g   8100.0  71522.0    4.8   
2            CJ 미정당국물떡볶이401.2g   2980.0  19879.0    4.8   
3   풀무원 말랑말랑 밀누들 떡볶이(국물떡볶이 소스)   4700.0  23586.0    4.8   
5         피코크 BT21 매운치즈떡볶이605g   6980.0    394.0    4.5   
6     CJ 미정당 매콤까르보나라 누들떡볶이400g   4980.0   1822.0    4.8   
7            CJ 미정당순쌀떡볶이401.2g   2980.0   1800.0    4.8   
8         [노브랜드] 간장 떡볶이 402.5g   2980.0   7849.0    4.6   
9                대림 로제떡볶이 343g   3980.0    131.0    4.4   
10        피코크 피콕분식 신당동식떡볶이970g   6980.0    398.0    4.6   
11               대림 국물떡볶이 373g   3480.0   1780.0    4.7   
12      청정원 야식이夜베이컨&로제떡볶이 485g   7980.0    119.0    4.8   
13               대림 짜장떡볶이 343g   3980.0     89.0    4.6   
14       풀무원 달콤고추장크림 누들떡볶이375g   4700.0   2905.0    4.7   
15          [청정원] 달인 왕떡볶이 420g   4980.0   1908.0    4.7   
16      동원 떡볶이의신 신당동 즉석쫄볶이397g   3980.0   1528.0    4.7 

In [182]:
driver = webdriver.Chrome(executable_path='chromedriver')
cc = 0

# for i in range(0,5,1):
for i in range(0,len(df1),1):
    try:
        URL = df1['link'][i]
    except:
        continue
    
    driver.get(URL)
    
    try:
        endpoint = int(re.sub(r'[^0-9]', '', driver.find_element_by_class_name("btn_last").get_attribute("onclick")))
    except:
        endpoint = int(df1['review'][i] / 10)

    # 최대 1만개의 리뷰만 수집하기 위함 (속도 저하의 문제)
    if(endpoint >= 100):
        endpoint = 100

        
    review_text = []

    print("end point : " + str(endpoint))

    for p in range(1,endpoint+1):

        driver.execute_script("fn_GoCommentPage('"+str(p)+"')")

        reviews = driver.find_element_by_id("cdtl_cmt_tbody")
        testis = reviews.text
        testisL1 = testis.split("\n")

        # 상품 기본정보 수집
        if(p==1):
            collection_point = "SSG"
            url = URL
            
            try:
                brand_name = driver.find_element_by_class_name("cdtl_ico_txt").text
            except:
                brand_name = ""
                
            product_name = df1['name'][i]
            product_price = df1['price'][i]
            
            cdtlInfo = re.findall('\(([^)]+)', driver.find_element_by_class_name("cdtl_txt_info").text)
            total_capacity = ""
            price_unit = ""
            
            if(len(cdtlInfo) > 0):
                
                cdtlInfoSplit = cdtlInfo[0].split(', ')

                total_capacity = cdtlInfoSplit[0].split(':')[1].strip()
                price_unit = cdtlInfoSplit[1].split(':')[1].strip()
  
                
            
            review_mean = df1['grade'][i]
            review_count = df1['review'][i]
            
            img_url = "http::" + df1['img_url'][i]
            
        time.sleep(1.0)

        for j in range(0,len(testisL1)):
            if(testisL1[j] == "" or testisL1[j] == "사진" or testisL1[j] == "비디오" ):
                continue;
            else:
                review_text.append(testisL1[j])
               
    result = pd.DataFrame(columns=['NO','INDEX','SCORE','REVIEW','REVIEW_ID','REVIEW_DATE'])
    for j in range(0,len(review_text),5):
        cc = cc + 1
        review_textL1 = review_text[j:j+5]
        review_textL1.insert(0,cc)
        resule_part = pd.DataFrame([review_textL1], columns=['NO','INDEX','SCORE','REVIEW','REVIEW_ID','REVIEW_DATE'])
        resule_part['SCORE'] = resule_part['SCORE'].str.replace('구매 고객 평점 별 5개 중', '').str.replace('개', '').astype(int)

        result = pd.concat([result,resule_part])
        
        
    cust_satfy = "http://210.178.19.87:9080/IMG/SERVICE2/" + product_name.replace(" ","") + ".png"
        
    ## bind ## 
    result = pd.concat([result, pd.DataFrame([collection_point],columns=["MARKET_PLATFORM"])], axis=1) 
    result = pd.concat([result, pd.DataFrame([url],columns=["PRODUCT_URL"])], axis=1) 
    result = pd.concat([result, pd.DataFrame([brand_name],columns=["BRAND_NAME"])], axis=1) 
    result = pd.concat([result, pd.DataFrame([product_name],columns=["PRODUCT_NAME"])], axis=1) 
    result = pd.concat([result, pd.DataFrame([product_price],columns=["PRODUCT_PRICE"])], axis=1) 
    result = pd.concat([result, pd.DataFrame([total_capacity],columns=["TOTAL_CAPACITY"])], axis=1)
    result = pd.concat([result, pd.DataFrame([price_unit],columns=["PRICE_FOR_UNIT"])], axis=1)
    result = pd.concat([result, pd.DataFrame([review_mean],columns=["PRODUCT_SCORE"])], axis=1)
    result = pd.concat([result, pd.DataFrame([review_count],columns=["REVIEW_COUNT"])], axis=1)
    result = pd.concat([result, pd.DataFrame([img_url],columns=["IMG_URL"])], axis=1)
    result = pd.concat([result, pd.DataFrame([cust_satfy],columns=["RES_IMG_URL"])], axis=1)
    ## bind ## 
        
    result.to_excel("./OUTPUT/SSG/"+df1['name'][i]+".xlsx")
    
    if(i == 0):
        result.to_csv("./OUTPUT/SERVICE2_RESULT_DATA.csv",index=False)
    else:
        result.to_csv("./OUTPUT/SERVICE2_RESULT_DATA.csv",mode='a', header=False,index=False)
    
driver.close()

end point : 100
end point : 100
end point : 100
end point : 100


KeyboardInterrupt: 

In [61]:
i

25

In [65]:
# 평점 전처리 함수
def star_preprocessing(value) :
#     value = int(text)

    if value <= 3 :
        return '0'
    else :
        return '1'

# 형태소 분석을 위한 함수
def tokenizer(text) :
    okt = Okt()
    return okt.morphs(text)

def step1_data_preprocessing() :
    # 수집한 데이터를 읽어온다.
    df = pd.read_csv('./감성분석입력자료/cos27.txt')

    # 전처리를 수행한다.
    df['rating'] = df['rating'].apply(star_preprocessing)

    # 학습데이터와 테스트 데이터로 나눈다.
    text_list = df['content'].tolist()
    star_list = df['rating'].tolist()

    from sklearn.model_selection import train_test_split

    # 80%는 학습, 20%는 test
    text_train, text_test, star_train, star_test = train_test_split(text_list, star_list, test_size=0.2, random_state=0)

    return text_train, text_test, star_train, star_test

def step2_learning(X_train, y_train, X_test, y_test):
    # 주어진 데이터를 단어 사전으로 만들고 각 단어의 빈도수를 계산한 후 벡터화 하는 객체 생성
    tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)

    # 문장별 나오는 단어수 세서 수치화, 벡터화해서 학습을 시킨다.
    logistic = LogisticRegression(C=10.0, penalty='l2', random_state=0)

    pipe = Pipeline([('vect', tfidf), ('clf', logistic)])

    # 학습한다.
    pipe.fit(X_train, y_train)

    # 학습 정확도 측정
    y_pred = pipe.predict(X_test)
    print(accuracy_score(y_test, y_pred))

    # 학습된 모델을 저장한다.
    with open('pipe.dat', 'wb') as fp :
        pickle.dump(pipe, fp)
        
    print('저장완료')
    
    
def step3_using_model(text) :
    # 객체를 복원한다.
    with open('pipe.dat', 'rb') as fp:
        pipe = pickle.load(fp)

    str = [text]
    # 예측 정확도
    r1 = np.max(pipe.predict_proba(str) * 100)
    # 예측 결과
    r2 = pipe.predict(str)[0]
    
    res = [r1,r2]
    #print('정확도 : %.3f' % r1)
    
    return res

        
# 학습 함수
def learing() :
    text_train, text_test, star_train, star_test = step1_data_preprocessing()
    step2_learning(text_train, star_train, text_test, star_test)

# 사용 함수
def using(inputtext) :
    res = step3_using_model(inputtext)
    return res

In [31]:
learing() # 학습 수행

0.8
저장완료


In [66]:
keyword = ["./OUTPUT/SERVICE2_RESULT_DATA.csv"]

for k in keyword:

    print(k)
    # 파일을 읽기
    data = pd.read_csv(k)

    data['RES_PN'] = ""
    data['RES_POS_RATE'] = ""
    data['RES_NEG_RATE'] = ""
    content = data['REVIEW']
    
    count = -1
    
    for i in content:
        
        count = count + 1
        res1 = using(i)
        
        if (res1[1] == "1"):
            data['RES_PN'][count] = "긍정"
            data['RES_POS_RATE'][count] = res1[0] 
            data['RES_NEG_RATE'][count] = 100 - res1[0] 
        else:
            data['RES_PN'][count] = "부정"
            data['RES_POS_RATE'][count] = 100 - res1[0] 
            data['RES_NEG_RATE'][count] = res1[0] 

        
    data.to_csv("./OUTPUT/SERVICE2_RESULT_DATA.csv",index=False)
    

./OUTPUT/SERVICE2_RESULT_DATA.csv


C:\Users\saima\AppData\Local\Temp/ipykernel_13736/2597522525.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['RES_PN'][count] = "긍정"
C:\Users\saima\AppData\Local\Temp/ipykernel_13736/2597522525.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['RES_POS_RATE'][count] = res1[0]
C:\Users\saima\AppData\Local\Temp/ipykernel_13736/2597522525.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['RES_NEG_RATE'][count] = 

In [177]:
keyword = ['./OUTPUT/SERVICE2_RESULT_DATA.csv']
import matplotlib as mpl
mpl.rcParams['font.size'] = 24
plt.rcParams['savefig.facecolor']='white'

for k in keyword:

    # 파일을 읽기
    data = pd.read_csv('./OUTPUT/SERVICE2_RESULT_DATA.csv')
    
    productlist = pd.unique(data["PRODUCT_NAME"])
    
    for p in productlist:
        
        dataL1 = data[data["PRODUCT_NAME"] == p]
        
        ratio = [len(dataL1[dataL1['RES_PN'] == '긍정']),len(dataL1[dataL1['RES_PN'] == '부정'])]
        labels = ['긍정','부정']
        colors = ['lightskyblue', 'lightcoral']
        plt.pie(ratio, labels=labels, autopct='%.1f%%',colors=colors)
        plt.title(p + " 긍부정 비율",fontsize=28)
        pp = str(p).replace(" ","")
        pp = str(pp).replace('[','')
        pp = str(pp).replace(']','')
        print(pp)
        plt.savefig("./긍부정분석결과이미지/"+pp+".png",dpi=600)
        plt.close()

    

풀무원국물떡볶이423.5g(2인분)
노브랜드떡볶이402.5g
CJ미정당국물떡볶이401.2g
풀무원말랑말랑밀누들떡볶이(국물떡볶이소스)
피코크BT21매운치즈떡볶이605g
CJ미정당매콤까르보나라누들떡볶이400g
CJ미정당순쌀떡볶이401.2g
노브랜드간장떡볶이402.5g
대림로제떡볶이343g
피코크피콕분식신당동식떡볶이970g
대림국물떡볶이373g
청정원야식이夜베이컨&로제떡볶이485g
대림짜장떡볶이343g
풀무원달콤고추장크림누들떡볶이375g
청정원달인왕떡볶이420g
동원떡볶이의신신당동즉석쫄볶이397g
청정원달인기름떡볶이420g
농심쿡탐짜파게티맛국물떡볶이380g
동원떡볶이의신즉석라볶이404g
동원떡볶이의신학교앞국물떡볶이372g
풀무원누들어묵볶이국물떡볶이소스423.5g
동원떡볶이의신짜장라볶이372g
뽀로로짜장떡볶이110g
뽀로로크림떡볶이115g
풀무원밀누들맵칼떡볶이373.5g
맛있는국물떡볶이424g
풀무원양송이버섯크림누들떡볶이375g
청정원달인국물떡볶이420g
제천빨간오뎅떡볶이세트(646g)
프레시지호로록우삼겹떡볶이2인분
미로식당국물떡볶이460g
추억의국민학교떡볶이매운맛
CJ미정당어묵떡볶이346g
추억의국민학교떡볶이매콤짜장맛
CJ미정당비엔나떡볶이300g
CJ미정당스팸누들떡볶이520g
떡볶이,튀김,순대,분식류


In [155]:
df2 = pd.read_excel("./OUTPUT/SERVICE2_RESULT_DATA.xlsx")

In [156]:
df2['REG_DATE'] = ""
df2['MOD_DATE'] = ""
df2['REG_DATE'] = str(datetime.datetime.now())[0:10]
df2['MOD_DATE'] = str(datetime.datetime.now())[0:10]

In [160]:
df2['RES_IMG_URL'] = df2['RES_IMG_URL'].str.replace(r'\[','')
df2['RES_IMG_URL'] = df2['RES_IMG_URL'].str.replace(r'\]','')

C:\Users\saima\AppData\Local\Temp/ipykernel_13736/3347964585.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['RES_IMG_URL'] = df2['RES_IMG_URL'].str.replace(r'\[','')
C:\Users\saima\AppData\Local\Temp/ipykernel_13736/3347964585.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['RES_IMG_URL'] = df2['RES_IMG_URL'].str.replace(r'\]','')


In [181]:
df2['RES_IMG_URL'][2222]

'http://210.178.19.87:9080/IMG/SERVICE2/CJ미정당국물떡볶이401.2g.png'

In [163]:
df2

,NO,INDEX,SCORE,REVIEW,REVIEW_ID,REVIEW_DATE,MARKET_PLATFORM,PRODUCT_URL,BRAND_NAME,PRODUCT_NAME,...,PRICE_FOR_UNIT,PRODUCT_SCORE,REVIEW_COUNT,IMG_URL,RES_IMG_URL,RES_PN,RES_POS_RATE,RES_NEG_RATE,REG_DATE,MOD_DATE
0,1,60950,5,야채넣고 어묵넣고 ?들어있는 소스넣고 ?참고로 소스부족으로 인해 고추장 나머지양념추...,kin*******,2021-05-10,SSG,http://www.ssg.com/item/itemView.ssg?itemId=00...,NaN,[풀무원] 국물 떡볶이 423.5g (2인분),...,423g,4.9,60929,http:://item.ssgcdn.com/88/68/98/item/00000089...,http://210.178.19.87:9080/IMG/SERVICE2/풀무원국물떡볶...,긍정,62.099157,37.900843,2021-09-12,2021-09-12
1,2,60949,5,자극적이지 않은 풀무원 국물떡볶이!! (2인분이지만 맛있어서 혼자 다먹음) 풀무원 ...,kdh*******,2020-09-07,SSG,http://www.ssg.com/item/itemView.ssg?itemId=00...,NaN,[풀무원] 국물 떡볶이 423.5g (2인분),...,423g,4.9,60929,http:://item.ssgcdn.com/88/68/98/item/00000089...,http://210.178.19.87:9080/IMG/SERVICE2/풀무원국물떡볶...,긍정,77.019281,22.980719,2021-09-12,2021-09-12
2,3,60948,5,편의점에서 파는 국물떡볶이만 먹어봤는데 그것도 맛나서 이 제품도 고민없이 구매해 보...,coc*******,2021-04-12,SSG,http://www.ssg.com/item/itemView.ssg?itemId=00...,NaN,[풀무원] 국물 떡볶이 423.5g (2인분),...,423g,4.9,60929,http:://item.ssgcdn.com/88/68/98/item/00000089...,http://210.178.19.87:9080/IMG/SERVICE2/풀무원국물떡볶...,긍정,66.210872,33.789128,2021-09-12,2021-09-12
3,4,60947,5,평소 자주 애용하는 풀무원국물떡볶이예요. 매번 구입하기 번거로워서 이번엔 2봉지 구...,jel*******,2020-09-22,SSG,http://www.ssg.com/item/itemView.ssg?itemId=00...,NaN,[풀무원] 국물 떡볶이 423.5g (2인분),...,423g,4.9,60929,http:://item.ssgcdn.com/88/68/98/item/00000089...,http://210.178.19.87:9080/IMG/SERVICE2/풀무원국물떡볶...,긍정,71.617283,28.382717,2021-09-12,2021-09-12
4,5,60946,5,매번 쓱배송으로 집에서 편하게 싱싱한 제품들받을수 있어서,ae1*******,2021-09-01,SSG,http://www.ssg.com/item/itemView.ssg?itemId=00...,NaN,[풀무원] 국물 떡볶이 423.5g (2인분),...,423g,4.9,60929,http:://item.ssgcdn.com/88/68/98/item/00000089...,http://210.178.19.87:9080/IMG/SERVICE2/풀무원국물떡볶...,긍정,78.661988,21.338012,2021-09-12,2021-09-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26685,26686,843,5,첨 먹어봤는데 맛있어요,aca*******,2021-01-28,SSG,http://www.ssg.com/item/dealItemView.ssg?itemI...,NaN,"떡볶이, 튀김, 순대, 분식류",...,400g,4.8,1822,http:://item.ssgcdn.com/85/99/09/item/10001120...,"http://210.178.19.87:9080/IMG/SERVICE2/떡볶이,튀김,...",부정,40.721333,59.278667,2021-09-12,2021-09-12
26686,26687,842,5,일상적으로 먹기 조음,kan*******,2021-01-27,SSG,http://www.ssg.com/item/dealItemView.ssg?itemI...,NaN,"떡볶이, 튀김, 순대, 분식류",...,400g,4.8,1822,http:://item.ssgcdn.com/85/99/09/item/10001120...,"http://210.178.19.87:9080/IMG/SERVICE2/떡볶이,튀김,...",긍정,73.472981,26.527019,2021-09-12,2021-09-12
26687,26688,841,5,"가성비 좋고 맛도 좋아요, 무료배송 너무 좋네요~~",ron*******,2021-01-25,SSG,http://www.ssg.com/item/dealItemView.ssg?itemI...,NaN,"떡볶이, 튀김, 순대, 분식류",...,400g,4.8,1822,http:://item.ssgcdn.com/85/99/09/item/10001120...,"http://210.178.19.87:9080/IMG/SERVICE2/떡볶이,튀김,...",긍정,97.764605,2.235395,2021-09-12,2021-09-12
26688,26689,840,5,까르보나라 맛이 좋았어요,art*******,2021-01-24,SSG,http://www.ssg.com/item/dealItemView.ssg?itemI...,NaN,"떡볶이, 튀김, 순대, 분식류",...,400g,4.8,1822,http:://item.ssgcdn.com/85/99/09/item/10001120...,"http://210.178.19.87:9080/IMG/SERVICE2/떡볶이,튀김,...",긍정,80.106568,19.893432,2021-09-12,2021-09-12


In [166]:
df2.to_csv("./RES.csv",index=False)
df2.to_excel("./RES.xlsx",index=False)